# Generate Bingo Cards

In [1]:
import random
import pandas as pd

In [2]:
# ========================================
# CONFIGURATION
# ========================================

PHRASES = [
    "entry 01",
    "entry 02",
    "Graph too small to read",
    "entry 03",
    "Applying for more funding",
    "Slide with too much text",
    "Preliminary data",
    "We'll publish this soon",
    "Projector or slideshow issue",
    "entry 04",
    "entry 05",
    "Any questions? (silence)",
    "entry 06",
    "AI or machine learning mentioned",
    "entry 07",
    "entry 08",
    "I'll keep it short",
    "Slide includes a meme",
    "entry 09",
    "entry 10",
    "entry 11",
    "entry 12",
    "Someone wears a Festive sweater",
    "entry 13"
]

N_CARDS = 50
FREE_POS = (2, 2)



In [3]:
# ========================================
# FUNCTIONS
# ========================================

def make_card(phrases):
    """Generate one 5x5 bingo card with a FREE center."""
    shuffled = random.sample(phrases, len(phrases))
    grid = [["" for _ in range(5)] for _ in range(5)]
    idx = 0
    for r in range(5):
        for c in range(5):
            if (r, c) == FREE_POS:
                grid[r][c] = "FREE"
            else:
                grid[r][c] = shuffled[idx]
                idx += 1
    return grid


def get_winning_lines(card):
    """Return all 12 winning line sets (as frozensets of phrases)."""
    lines = []
    # Rows
    for r in range(5):
        lines.append(frozenset(card[r]))
    # Columns
    for c in range(5):
        col = [card[r][c] for r in range(5)]
        lines.append(frozenset(col))
    # Diagonals
    diag1 = [card[i][i] for i in range(5)]
    diag2 = [card[i][4 - i] for i in range(5)]
    lines.append(frozenset(diag1))
    lines.append(frozenset(diag2))
    return lines

In [4]:
# ========================================
# CARD GENERATION WITH UNIQUE WINNING LINES
# ========================================

cards = []
unique_lines = set()
attempts = 0

while len(cards) < N_CARDS:
    attempts += 1
    card = make_card(PHRASES)
    lines = get_winning_lines(card)
    # Check if any line duplicates an existing one
    if any(line in unique_lines for line in lines):
        continue  # discard this card and try again
    # Add all its lines to the global set
    unique_lines.update(lines)
    cards.append(card)

print(f"Generated {len(cards)} unique cards after {attempts} attempts.")
print(f"Total unique winning lines stored: {len(unique_lines)}")

# ========================================
# SAVE CARDS
# ========================================

for i, card in enumerate(cards, 1):
    df = pd.DataFrame(card)
    df.to_csv(f"./bingo_cards/bingo_card_{i:02d}.csv", index=False, header=False)

print("Saved all cards as bingo_card_XX.csv files.")

Generated 50 unique cards after 59 attempts.
Total unique winning lines stored: 600
Saved all cards as bingo_card_XX.csv files.


# Create PDFs

In [5]:
from reportlab.lib.pagesizes import A4
from reportlab.lib import colors
from reportlab.lib.units import cm
from reportlab.pdfgen import canvas

import glob


In [6]:
# ========================================
# CONFIGURATION
# ========================================
CARD_SIZE = 5
CELL_SIZE = 1.8 * cm
FONT_SIZE = 8
TITLE_FONT_SIZE = 14
OUTPUT_PDF = "bingo_cards_two_per_page_fixed.pdf"

# Colors
LINE_COLOR = colors.darkgreen
FREE_COLOR = colors.lightgreen
TEXT_COLOR = colors.black
TITLE_COLOR = colors.darkred

# Left side text
LEFT_TEXT = [
    "Welcome to the Institute Assembly Bingo!",
    "",
    "  .-''-.",
    " /,..___\ ",
    "() {_____}",
    "  (/-@-@-\)",
    "  {`-=^=-'}",
    "  {  `-'  } ",
    "   {     }",
    "    `---'",
    "",
    "Instructions:",
    "- Listen carefully during the assembly.",
    "- Check your boxes on the right-side",
    "  card as they are mentioned.",
    "- First one to get five in a row shouts 'BINGO!'",
    "",
    "Win the extraordinaire privilege to chose ",
    "your Secret Santa gift first!",
    "Have fun and enjoy the Christmas party!"
]

FREE_POS = (2, 2)  # center cell

# Horizontal positions
LEFT_X = 1.5 * cm
RIGHT_X = 11 * cm  # right half for bingo card

# Vertical positions for top and bottom cards
PAGE_HEIGHT = A4[1]
TOP_Y = PAGE_HEIGHT - 4 * cm  # start 4cm from top
BOTTOM_Y = PAGE_HEIGHT / 2 - 1 * cm



<>:21: SyntaxWarning: invalid escape sequence '\ '
<>:23: SyntaxWarning: invalid escape sequence '\)'
<>:21: SyntaxWarning: invalid escape sequence '\ '
<>:23: SyntaxWarning: invalid escape sequence '\)'
/tmp/ipykernel_2351576/203853623.py:21: SyntaxWarning: invalid escape sequence '\ '
  " /,..___\ ",
/tmp/ipykernel_2351576/203853623.py:23: SyntaxWarning: invalid escape sequence '\)'
  "  (/-@-@-\)",


In [7]:
# ========================================
# LOAD CARDS
# ========================================
card_files = sorted(glob.glob("./bingo_cards/bingo_card_*.csv"))
cards = [pd.read_csv(f, header=None).values.tolist() for f in card_files]
print(f"Loaded {len(cards)} cards for PDF output.")

Loaded 50 cards for PDF output.


In [8]:
# ========================================
# DRAW FUNCTIONS
# ========================================
def draw_card(c, card, x0, y0, card_num):
    # Draw title above the grid
    c.setFont("Helvetica-Bold", TITLE_FONT_SIZE)
    c.setFillColor(TITLE_COLOR)
    c.drawCentredString(x0 + CARD_SIZE*CELL_SIZE/2, y0 + CARD_SIZE*CELL_SIZE + 0.5*cm, f"BINGO #{card_num}")

    # Draw grid
    c.setStrokeColor(LINE_COLOR)
    for i in range(CARD_SIZE + 1):
        c.line(x0, y0 + i*CELL_SIZE, x0 + CARD_SIZE*CELL_SIZE, y0 + i*CELL_SIZE)
        c.line(x0 + i*CELL_SIZE, y0, x0 + i*CELL_SIZE, y0 + CARD_SIZE*CELL_SIZE)

    # Fill cells
    for r in range(CARD_SIZE):
        for col in range(CARD_SIZE):
            text = card[r][col]
            cx = x0 + (col + 0.5)*CELL_SIZE
            cy = y0 + (r + 0.5)*CELL_SIZE

            if text == "FREE":
                c.setFillColor(FREE_COLOR)
                c.rect(x0 + col*CELL_SIZE, y0 + r*CELL_SIZE, CELL_SIZE, CELL_SIZE, fill=1, stroke=0)
                c.setFillColor(TEXT_COLOR)

            # Wrap text
            words = text.split()
            lines = []
            line = ""
            for w in words:
                if len(line + " " + w) > 10:
                    lines.append(line.strip())
                    line = w
                else:
                    line += " " + w
            if line:
                lines.append(line.strip())

            # Adjust font size for many lines
            base_font = FONT_SIZE
            while len(lines) > 3 and base_font > 6:
                base_font -= 1
            c.setFont("Helvetica", base_font)

            # Vertical centering
            line_height = base_font * 1.1
            total_height = line_height * len(lines)
            start_y = cy - total_height / 2 + line_height * 0.3
            for i, line in enumerate(lines):
                c.setFillColor(TEXT_COLOR)
                c.drawCentredString(cx, start_y + (len(lines)-1-i)*line_height, line)


def draw_left_text(c, x0, y0, text_lines):
    c.setFont("Courier", 7)
    c.setFillColor(TEXT_COLOR)
    line_height = 12
    for i, line in enumerate(text_lines):
        c.drawString(x0, y0 + (len(text_lines)-1-i)*line_height, line)


In [9]:
# ========================================
# CREATE PDF
# ========================================
c = canvas.Canvas(OUTPUT_PDF, pagesize=A4)

i = 0
while i < len(cards):
    # Top card
    draw_left_text(c, LEFT_X, TOP_Y - CELL_SIZE*CARD_SIZE + 0*cm, LEFT_TEXT)
    draw_card(c, cards[i], RIGHT_X, TOP_Y - CELL_SIZE*CARD_SIZE - 0.5*cm, i+1)
    i += 1

    # Bottom card
    if i < len(cards):
        draw_left_text(c, LEFT_X, BOTTOM_Y - CELL_SIZE*CARD_SIZE + 0*cm, LEFT_TEXT)
        draw_card(c, cards[i], RIGHT_X, BOTTOM_Y - CELL_SIZE*CARD_SIZE - 0.5*cm, i+1)
        i += 1

    c.showPage()

c.save()
print(f"Saved printable PDF with {len(cards)} cards → {OUTPUT_PDF}")


Saved printable PDF with 50 cards → bingo_cards_two_per_page_fixed.pdf
